In [1]:
%matplotlib inline
from skimage import data, io, filters, exposure
from skimage.segmentation import slic, felzenszwalb
from skimage.transform import resize
from skimage.morphology import square, closing, binary_closing
from matplotlib import pyplot as plt
from skimage import color
import numpy as np
from scipy import ndimage as ndi
from skimage.measure import moments_hu, moments, moments_central, moments_normalized

In [2]:
def get_feature(name):
    sample = io.imread(str(name))
    sample = resize(sample, (148,224), mode='constant')

    sample_hsv = color.convert_colorspace(sample,'RGB','HSV')
    
    # change green color to white, other color to black
    for i in range(sample_hsv.shape[0]):
        for j in range(sample_hsv.shape[1]):
            if sample_hsv[i,j,0] > 0.2083 and sample_hsv[i,j,0] < 0.4167 and sample_hsv[i,j,2]>0.1  and sample_hsv[i,j,1]>0.1:
                sample_hsv[i,j,1] = 0
                sample_hsv[i,j,2] = 1
            else:
                sample_hsv[i,j,2] = 0

    sample_rgb = color.convert_colorspace(sample_hsv,'HSV','RGB')

    sample_grey = color.rgb2grey(sample_rgb)

    # find center of the image
    t = 0
    res_i = 0
    res_j = 0
    for i in range(sample_grey.shape[0]-100):
        for j in range(sample_grey.shape[1]-100):
            if np.sum(sample_grey[i:i+100,j:j+100]) > t:
                res_i = i
                res_j = j
                t = np.sum(sample_grey[i:i+100,j:j+100])

    sample_cropped = sample_grey[res_i:res_i+100,res_j:res_j+100]

    closed = binary_closing(sample_cropped, square(2))

    closed = np.asarray(closed, dtype=np.double)

    M = moments(closed)
    cr = M[1, 0] / M[0, 0]
    cc = M[0, 1] / M[0, 0]

    mu = moments_central(closed, center=(cr,cc))
    nu = moments_normalized(mu)
    return moments_hu(nu)

In [3]:
feature = np.empty((0,8), int)

from os import listdir
from os.path import isfile, join
mypath = 'train_1'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for o in onlyfiles:
    feature = np.vstack((feature, np.hstack((get_feature(join(mypath,o)),0))))

/home/rian/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [4]:
mypath = 'train_2'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for o in onlyfiles:
    feature = np.vstack((feature,np.hstack((get_feature(join(mypath,o)),1))))

/home/rian/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [5]:
feature_test = np.empty((0,8), int)
mypath = 'test'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

x = 0
for o in onlyfiles:
    x = int(o[8:10])
    if x<67:
        u=0
    else:
        u=1
    feature_test = np.vstack((feature_test,np.hstack((get_feature(join(mypath,o)),u))))
    x += 1
feature_test.shape

/home/rian/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


(21, 8)

In [6]:
from sklearn.neural_network import MLPClassifier
# 15 0.6 50
# 9-3, 200, 0.1, 1500
mlp = MLPClassifier(hidden_layer_sizes=(9,3),
                    solver='sgd',
                    batch_size=200,
                    activation='relu',
                    learning_rate='adaptive',
                    learning_rate_init=0.1,
                    max_iter=1500,
                    momentum=0,
                    tol=0.0,
                    alpha=0,
                    verbose=True,
                    random_state=123)
train_x = feature[:,:-1]
train_y = feature[:,-1]
test_x = feature_test[:,:-1]
test_y = feature_test[:,-1]
test_x.shape, test_y.shape

mlp.fit(train_x, train_y)
print(mlp.score(test_x,test_y))
print(mlp.predict(test_x))

/home/rian/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:358: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn("Got `batch_size` less than 1 or larger than "


Iteration 1, loss = 0.70248258
Iteration 2, loss = 0.70071492
Iteration 3, loss = 0.69917939
Iteration 4, loss = 0.69784753
Iteration 5, loss = 0.69669407
Iteration 6, loss = 0.69569658
Iteration 7, loss = 0.69483521
Iteration 8, loss = 0.69409242
Iteration 9, loss = 0.69345274
Iteration 10, loss = 0.69290255
Iteration 11, loss = 0.69242991
Iteration 12, loss = 0.69202436
Iteration 13, loss = 0.69167675
Iteration 14, loss = 0.69137910
Iteration 15, loss = 0.69112447
Iteration 16, loss = 0.69090684
Iteration 17, loss = 0.69072097
Iteration 18, loss = 0.69056235
Iteration 19, loss = 0.69042707
Iteration 20, loss = 0.69031176
Iteration 21, loss = 0.69021353
Iteration 22, loss = 0.69012987
Iteration 23, loss = 0.69005866
Iteration 24, loss = 0.68999805
Iteration 25, loss = 0.68994647
Iteration 26, loss = 0.68990258
Iteration 27, loss = 0.68986523
Iteration 28, loss = 0.68983343
Iteration 29, loss = 0.68980636
Iteration 30, loss = 0.68978330
Iteration 31, loss = 0.68976364
Iteration 32, los

Iteration 356, loss = 0.68934061
Iteration 357, loss = 0.68933950
Iteration 358, loss = 0.68933839
Iteration 359, loss = 0.68933727
Iteration 360, loss = 0.68933616
Iteration 361, loss = 0.68933504
Iteration 362, loss = 0.68933393
Iteration 363, loss = 0.68933281
Iteration 364, loss = 0.68933169
Iteration 365, loss = 0.68933057
Iteration 366, loss = 0.68932945
Iteration 367, loss = 0.68932833
Iteration 368, loss = 0.68932721
Iteration 369, loss = 0.68932609
Iteration 370, loss = 0.68932497
Iteration 371, loss = 0.68932384
Iteration 372, loss = 0.68932272
Iteration 373, loss = 0.68932159
Iteration 374, loss = 0.68932047
Iteration 375, loss = 0.68931934
Iteration 376, loss = 0.68931821
Iteration 377, loss = 0.68931708
Iteration 378, loss = 0.68931595
Iteration 379, loss = 0.68931482
Iteration 380, loss = 0.68931369
Iteration 381, loss = 0.68931256
Iteration 382, loss = 0.68931142
Iteration 383, loss = 0.68931029
Iteration 384, loss = 0.68930915
Iteration 385, loss = 0.68930802
Iteration 

Iteration 648, loss = 0.68897523
Iteration 649, loss = 0.68897383
Iteration 650, loss = 0.68897242
Iteration 651, loss = 0.68897101
Iteration 652, loss = 0.68896960
Iteration 653, loss = 0.68896819
Iteration 654, loss = 0.68896677
Iteration 655, loss = 0.68896536
Iteration 656, loss = 0.68896394
Iteration 657, loss = 0.68896253
Iteration 658, loss = 0.68896111
Iteration 659, loss = 0.68895969
Iteration 660, loss = 0.68895827
Iteration 661, loss = 0.68895685
Iteration 662, loss = 0.68895543
Iteration 663, loss = 0.68895400
Iteration 664, loss = 0.68895258
Iteration 665, loss = 0.68895115
Iteration 666, loss = 0.68894972
Iteration 667, loss = 0.68894829
Iteration 668, loss = 0.68894686
Iteration 669, loss = 0.68894543
Iteration 670, loss = 0.68894400
Iteration 671, loss = 0.68894257
Iteration 672, loss = 0.68894113
Iteration 673, loss = 0.68893970
Iteration 674, loss = 0.68893826
Iteration 675, loss = 0.68893682
Iteration 676, loss = 0.68893538
Iteration 677, loss = 0.68893394
Iteration 

Iteration 985, loss = 0.68842495
Iteration 986, loss = 0.68842306
Iteration 987, loss = 0.68842117
Iteration 988, loss = 0.68841927
Iteration 989, loss = 0.68841738
Iteration 990, loss = 0.68841548
Iteration 991, loss = 0.68841358
Iteration 992, loss = 0.68841168
Iteration 993, loss = 0.68840978
Iteration 994, loss = 0.68840787
Iteration 995, loss = 0.68840597
Iteration 996, loss = 0.68840406
Iteration 997, loss = 0.68840215
Iteration 998, loss = 0.68840024
Iteration 999, loss = 0.68839832
Iteration 1000, loss = 0.68839641
Iteration 1001, loss = 0.68839449
Iteration 1002, loss = 0.68839257
Iteration 1003, loss = 0.68839065
Iteration 1004, loss = 0.68838873
Iteration 1005, loss = 0.68838681
Iteration 1006, loss = 0.68838488
Iteration 1007, loss = 0.68838295
Iteration 1008, loss = 0.68838103
Iteration 1009, loss = 0.68837909
Iteration 1010, loss = 0.68837716
Iteration 1011, loss = 0.68837523
Iteration 1012, loss = 0.68837329
Iteration 1013, loss = 0.68837135
Iteration 1014, loss = 0.6883

Iteration 1348, loss = 0.68760695
Iteration 1349, loss = 0.68760427
Iteration 1350, loss = 0.68760159
Iteration 1351, loss = 0.68759891
Iteration 1352, loss = 0.68759623
Iteration 1353, loss = 0.68759354
Iteration 1354, loss = 0.68759085
Iteration 1355, loss = 0.68758816
Iteration 1356, loss = 0.68758547
Iteration 1357, loss = 0.68758277
Iteration 1358, loss = 0.68758007
Iteration 1359, loss = 0.68757737
Iteration 1360, loss = 0.68757466
Iteration 1361, loss = 0.68757196
Iteration 1362, loss = 0.68756925
Iteration 1363, loss = 0.68756653
Iteration 1364, loss = 0.68756382
Iteration 1365, loss = 0.68756110
Iteration 1366, loss = 0.68755838
Iteration 1367, loss = 0.68755565
Iteration 1368, loss = 0.68755293
Iteration 1369, loss = 0.68755020
Iteration 1370, loss = 0.68754746
Iteration 1371, loss = 0.68754473
Iteration 1372, loss = 0.68754199
Iteration 1373, loss = 0.68753925
Iteration 1374, loss = 0.68753651
Iteration 1375, loss = 0.68753376
Iteration 1376, loss = 0.68753101
Iteration 1377

/home/rian/.local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
